### Importing the required Libraries 

In [ ]:
import pandas as pd
import numpy as np
import re

### Requesting for the name of the file to be processed 

In [ ]:
dataset = input('Enter the name of your dataset with the extension: ')

### Using Regular expression to access the file and determine what kind of file it is

In [ ]:
result = re.findall(r'[a-z]+',dataset)

if result[1] == 'csv':
    frame = pd.read_csv(dataset)
    print('The dataset was succesfully imported \n')
    first_5rows = frame.head()
    last_5rows = frame.tail()
    print('The first five rows are :' ,first_5rows)
    print('The last five rows are :',last_5rows)
elif result[1] == 'xlsx':
    frame = pd.read_excel(dataset)
    print('The dataset was succesfully imported \n')
    first_5rows = frame.head()
    last_5rows = frame.tail()
    print('The first five rows are :' ,first_5rows)
    print('The last five rows are :',last_5rows)

### Splitting the data into Categorical and Continuous datasets

In [ ]:
catdf = frame.select_dtypes(include = ['object'])
catdrop = list(catdf.columns)
condf = frame.drop(catdrop,axis = 1)

### Processing the Continous dataset

In [ ]:
if condf.empty == True:
    print('The dataset does not contain any continuous features')
else:
    confeat = list(condf.columns)
    conHead =  ['Count', 'Miss %', 'Card.', 'Min', '1st Qrt.', 'Mean', 'Median', '3rd Qrt', 'Max', 'Standard Deviation'] #Creating the features to be reported
    
    conOutDf = pd.DataFrame(index = confeat, columns = conHead) #Created the data Quality report dataframe
    conOutDf.index.name = 'FeatureName'
    
    #Populating the count data
    counting = condf.count()
    conOutDf[conHead[0]] = counting 
    
    #Missing Percentage
    percent = ['']* len(confeat)
    for j in condf:
        if condf[j].isnull().values.any()==False:
            percent [confeat.index(j)]=0.00
        else:
            nu = condf[j].isnull().sum()
            co = condf[j].count()
            per = nu/co *100
            percent[confeat.index(j)] = per
    conOutDf[conHead[1]] = percent  #This adds the percentage missing to the report 
    
    #Cardinality
    conOutDf[conHead[2]] = condf.nunique()  
    #Minimum 
    conOutDf[conHead[3]] = condf.min()
    #1st Quartile
    conOutDf[conHead[4]] = condf.quantile(0.25)
    #Mean
    conOutDf[conHead[5]] = round(condf.mean(),2)
    #Median
    conOutDf[conHead[6]] = condf.median()
    #3rd Quartile
    conOutDf[conHead[7]] = condf.quantile(0.75)
    #Max
    conOutDf[conHead[8]] = condf.max()
    #Standard Deviation
    conOutDf[conHead[9]] = round(condf.std(),2)



''' Collecting the outliers for each continuous feature'''
outliers =[]

third_max = conOutDf['3rd Qrt'] - conOutDf['Max']
third_median = conOutDf['3rd Qrt']-conOutDf['Median'] 

first_min = conOutDf['1st Qrt.'] - conOutDf['Min']
first_median = conOutDf['1st Qrt.']-conOutDf['Median']   

leg = range(len(third_max)) 

for j in leg:
    if third_max[j] > third_median[j] or first_min[j] >first_median[j] :  #This basically checks  the difference between the variables
        outliers.append(conOutDf.index[j])

print('The features with outliers: ',outliers)

### Processing the Categorical dataset

In [ ]:
if catdf.empty == True:
    print('The dataset does not contain any categorical features')
else:
    catfeat = list(catdf.columns)
    catHead = ['Count', 'Miss %', 'Card.', 'Mode', 'Mode Freq', 'Mode %', '2nd Mode', '2nd Mode Freq', '2nd Mode %']
    catOutDf = pd.DataFrame(index = catfeat,columns = catHead) #Creates the Data quality report dataframe
    catOutDf.index.name = 'FeatureName'
    
    #Count
    counting = catdf.count()
    catOutDf[catHead[0]] = counting
    #Missing Percent
    percent = ['']*len(catfeat)
    for k in catdf:
        if catdf[k].isnull().values.any() == False:
            percent[catfeat.index(k)] = 0.00
        else:
            nu = catdf[k].isnull().sum()
            po = catdf[k].count()
            per = nu/po *100
            percent[catfeat.index(k)] = per
    catOutDf[catHead[1]]= percent
    #Cardinality
    catOutDf[catHead[2]] = catdf.nunique()
    #Mode
    modethem = [''] *len(catfeat)
    modethem2 = ['']*len(catfeat)
    for shi in catdf:
        mo = catdf[shi].mode()
        values = catdf[shi].value_counts()
        mod = list(mo)                    #This makes a list of the highest frequency 
        modem = mod[0]
        modem2 = values.index[1]
        modethem[catfeat.index(shi)] = modem
        modethem2[catfeat.index(shi)] = modem2
    catOutDf[catHead[3]] = modethem
    catOutDf[catHead[6]] = modethem2
    
    #Mode Frequency 
    modefreq = ['']* len(catfeat)
    modefreq2 = ['']*len(catfeat)
    for j in catdf:
        values = catdf[j].value_counts() #This gives a series of the frequency 
        index = catfeat.index(j)       #This gives the location of the feature
        
        modee = values.index[0]
        modee2 = values.index[1]
        modethemfreq = values.loc[modee]
        modethemfreq2 = values.loc[modee2]
        modefreq[index] = modethemfreq
        modefreq2[index] = modethemfreq2
    catOutDf[catHead[4]] = modefreq
    catOutDf[catHead[7]] = modefreq2
    #Mode %
    mode_percentage = ['']*len(catfeat) #This creates an empty list
    mode_percentage2 = ['']*len(catfeat)
    for zen in catdf:
        values = catdf[zen].value_counts()
        index = catfeat.index(zen)
        mode = values.index[0]
        mode_percentage[index] = round((values[0]/counting[zen])*100,2)  #This gets the percentage and puts it in the empty list created
        mode_percentage2[index] = round((values[1]/counting[zen])*100,2)
    catOutDf[catHead[5]] = mode_percentage
    catOutDf[catHead[8]] = mode_percentage2

### Lastly outputting the results of the dataset into Continous and Categorical files 


In [ ]:
catOutDf.to_csv('Categorical_Data_Quality_Report.csv') #This would export to the directory set for the python environment

conOutDf.to_csv('Continuous_Data_Quality_Report.csv')